# ENUM Translator

In [114]:
import os
from dotenv import load_dotenv
import ollama

load_dotenv()
ollama_client = ollama.Client(host=os.getenv("SERVER_URL"))

In [115]:
model_names = [model.model for model in ollama_client.list().models]
print(model_names)


['llama3.1:8b', 'gemma3:12b', 'gemma3:4b', 'llama3.2:latest']


In [116]:
LLM_MODEL = model_names[0] #llama3.1:8b

In [142]:
def translation_agent(enum_names, toLang):
    system_prompt = (
    f"""You are an AI assistant specialized in translating ENUM names into human-readable messages.
    Your task is to take an ENUM written in English and perform the following:

    1. Convert the ENUM name into a clear, human-readable English sentence.
    2. Translate the resulting English sentence into {toLang}.

    ### Instructions:
    - Maintain proper capitalization and spacing.
    - The output must be a JSON object where:
      - The ENUM name is the key.
      - The value is an array containing two objects:
        - One for the English sentence (`"en"` key).
        - One for the {toLang} translation (`"{toLang}"` key).
    - Ensure translations are natural and contextually accurate.
    - Do not include explanations or formatting notes, only return the JSON.
    - Do not include special characters newlines etc.

    ### Example Input:
    Enum: `UserAccountLocked`

    ### Expected Output (Assume the language to translate to is greek):
    {{
      "UserAccountLocked": [
        {{"en": "The user account is locked."}},
        {{"el": "Ο λογαριασμός χρήστη είναι κλειδωμένος."}}
      ]
    }}

    Now, process the given list of ENUMs accordingly.
    """
    )

    user_prompt = (
        f"Translate the following ENUMs from english to {toLang}.\n"
        f"ENUMS: {', '.join(enum_names)}"
    )

    response = ollama_client.chat(
        model=LLM_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        format="json",
    )

    return json.loads(response['message']['content'])


In [148]:
import json
from IPython.display import display, JSON

enums = ["ThisIsAmazing","ThisIsBad"]
response = translation_agent(enums, "spanish")
display(JSON(response))

<IPython.core.display.JSON object>